In [1]:
from games_setup import *
from SBMLLint.common import constants as cn
from SBMLLint.common.molecule import Molecule, MoleculeStoichiometry
from SBMLLint.common.reaction import Reaction
from SBMLLint.games.som import SOM
from SBMLLint.common.simple_sbml import SimpleSBML

import collections
import itertools
import networkx as nx
import numpy as np
import pandas as pd
#
from SBMLLint.common.stoichiometry_matrix import StoichiometryMatrix
from SBMLLint.games.mesgraph import MESGraph
from SBMLLint.games.games_pp import GAMES_PP, SOMStoichiometry, SOMReaction, TOLERANCE
from SBMLLint.games.games_report import GAMESReport, SimplifiedReaction

ReactionOperation = collections.namedtuple("ReactionOperation", 
    "reaction operation")

Current Directory: /Users/woosubshin/Desktop/ModelEngineering/SBMLLint/notebook


In [2]:
# Model 248 is for canceling error
# Model 14/167 for a type three error / echelon
# Model 7 for type 1 error
# Model 18 for type 2 error

######## examine model 628 for large echelon error ###########
simple = load_file_from_curated_data(479)
# for r in simple.reactions:
#   if r.category != cn.REACTION_BOUNDARY and r.category != cn.REACTION_1_1:
#     print(r.makeIdentifier(is_include_kinetics=False))
m = GAMES_PP(simple)
m.analyze()
gr = GAMESReport(m)

We just analyzed the data...
Type I error:  []
Type II error:  []
Canceling error:  []
Echelon error:  [R39:  -> {P}]
Type III error:  [R38: {GaGDPP=GaGTP} -> {GaGDP}]
Type I-SOM error:  set()


In [3]:
#np.linalg.cond(m.lower_inverse)

In [4]:
len(simple.reactions)

44

In [16]:
resulting_series = gr.getResultingSeries("statPhosphorylation")

In [18]:
type(resulting_series

numpy.float64

In [11]:
type1_report, error_num = gr.reportTypeOneError(m.type_one_errors, explain_details=False)
print(type1_report)
print(error_num)

We detected a mass imbalance from the following reactions:

1. SHMTr: CH2FH4 -> FH4
2. CH2FH4toHCHO: CH2FH4 -> FH4 + HCHO
3. SHMT: FH4 + serine -> CH2FH4
4. HCHOtoCH2FH4: FH4 + HCHO -> CH2FH4


----------------------------------------------------------------------

We detected a mass imbalance from the following reactions:

1. SHMTr: CH2FH4 -> FH4
2. DHFReductase: FH2f -> FH4
3. TYMS: CH2FH4 + dUMP -> FH2f


----------------------------------------------------------------------

We detected a mass imbalance from the following reactions:

1. FFH2syn: FH2f -> FFH2
2. ATIC12: FFH2 + AICAR -> FH2f


----------------------------------------------------------------------

We detected a mass imbalance from the following reactions:

1. MTX1deg: MTX1b -> MTX1
2. MTX1off: MTX1b -> MTX1 + DHFRf
3. MTX1on: MTX1 + DHFRf -> MTX1b


----------------------------------------------------------------------

We detected a mass imbalance from the following reactions:

1. MTX2deg: MTX2b -> MTX2
2. MTX2off: 

In [22]:
type2_report, error_num = gr.reportTypeTwoError([m.type_two_errors[1]], explain_details=True)
print(type2_report)
print(error_num)

We detected a mass imbalance from the following reactions:

These uni-uni reactions created mass-equivalence.

1. FFH2syn: FH2f -> FFH2
2. SHMTr: CH2FH4 -> FH4
3. DHFReductase: FH2f -> FH4

----------------------------------------------------------------------

The following reactions create mass-inequality.

4. MTHFR: CH2FH4 + NADPH -> CH3FH4
5. MTR: CH3FH4 + homocysteine -> FH4

----------------------------------------------------------------------

Based on the reactions above, we have mass-equivalent pseudo reactions.

(pseudo 4.) MTHFR: {CH2FH4=FFH2=FH2f=FH4} + {NADPH} -> {CH3FH4}
(pseudo 5.) MTR: {homocysteine} + {CH3FH4} -> {CH2FH4=FFH2=FH2f=FH4}

----------------------------------------------------------------------

However, the above pseudo reactions imply the following inequalities:

{CH2FH4=FFH2=FH2f=FH4} < {CH3FH4} < {CH2FH4=FFH2=FH2f=FH4}

This indicates a mass conflict between reactions.

----------------------------------------------------------------------

-----------

In [26]:
type2_report[-255:]

'{CH2FH4=FFH2=FH2f=FH4} < {CH3FH4} < {CH2FH4=FFH2=FH2f=FH4}\n\nThis indicates a mass conflict between reactions.\n\n----------------------------------------------------------------------\n\n----------------------------------------------------------------------\n\n'

In [24]:
k = "{CH2FH4=FFH2=FH2f=FH4} < {CHOFH4} < {CH2FH4=FFH2=FH2f=FH4}\n\nThis indicates a mass conflict between reactions.\n\n----------------------------------------------------------------------\n\n----------------------------------------------------------------------\n\n"
len(k)

255

In [12]:
type2_report

'We detected a mass imbalance from the following reactions:\n\nThese uni-uni reactions created mass-equivalence.\n\n1. FFH2syn: FH2f -> FFH2\n2. SHMTr: CH2FH4 -> FH4\n3. DHFReductase: FH2f -> FH4\n\n----------------------------------------------------------------------\n\nThe following reactions create mass-inequality.\n\n4. HCOOHtoCHOFH4: FH4 + formate + ATP -> CHOFH4\n5. MTHFD: CH2FH4 + NADP -> CHOFH4\n6. ATIC7: CHOFH4 + AICAR -> FH4\n\n----------------------------------------------------------------------\n\nBased on the reactions above, we have mass-equivalent pseudo reactions.\n\n(pseudo 4.) HCOOHtoCHOFH4: {CH2FH4=FFH2=FH2f=FH4} + {formate} + {ATP} -> {CHOFH4}\n(pseudo 5.) MTHFD: {CH2FH4=FFH2=FH2f=FH4} + {NADP} -> {CHOFH4}\n(pseudo 6.) ATIC7: {CHOFH4} + {AICAR=FGAR} -> {CH2FH4=FFH2=FH2f=FH4}\n\n----------------------------------------------------------------------\n\nHowever, the above pseudo reactions imply the following inequalities:\n\n{CH2FH4=FFH2=FH2f=FH4} < {CHOFH4} < {CH2FH

In [3]:
canceling_report, error_num = gr.reportCancelingError(m.canceling_errors, explain_details=True)
print(canceling_report)
print(error_num)

We detected a mass imbalance from the following reactions:

1. OxidativePhosphorylation: 6.00 ADP + CTtis -> 6.00 ATP

2. ATPase: ATP -> ADP

*ATP and ADP have the same mass according to the above reaction


Therefore, they will result in empty product with zero mass:

OxidativePhosphorylation: CTtis -> 

This indicates a mass conflict between reactions.

----------------------------------------------------------------------

----------------------------------------------------------------------



**********************************************************************


[2]


In [4]:
canceling_report

'We detected a mass imbalance from the following reactions:\n\n1. OxidativePhosphorylation: 6.00 ADP + CTtis -> 6.00 ATP\n\n2. ATPase: ATP -> ADP\n\n*ATP and ADP have the same mass according to the above reaction\n\n\nTherefore, they will result in empty product with zero mass:\n\nOxidativePhosphorylation: CTtis -> \n\nThis indicates a mass conflict between reactions.\n\n----------------------------------------------------------------------\n\n----------------------------------------------------------------------\n\n\n\n**********************************************************************\n\n'

In [8]:
m.echelon_errors[0]

statPhosphorylation:  -> {species_test}

In [9]:
len(simple.reactions)

7

In [5]:
echelon_report, error_num = gr.reportEchelonError(m.echelon_errors, explain_details=True)
print(echelon_report)
print(error_num)


We detected a mass imbalance
:  -> P

from the following reactions.

1. R9: LRGabg -> LR + GaGTP + Gbg
2. R38: GaGDPP -> GaGDP + P
3. R39: GaGDP + Gbg -> Gabg
4. R3: LR + Gabg -> LRGabg

----------------------------------------------------------------------

These uni-uni reactions created mass-equivalence.
(The molecules within a curly bracket have the same atomic mass.)

5. R26: GaGTP -> GaGDPP;   {GaGTP=GaGDPP}

----------------------------------------------------------------------

Based on the reactions above, we have mass-equivalent pseudo reactions.
(pseudo 1.) R9: {LRGabg} -> {GaGDPP=GaGTP} + {Gbg} + {LR}
(pseudo 2.) R38: {GaGDPP=GaGTP} -> {GaGDP} + {P}
(pseudo 3.) R39: {GaGDP} + {Gbg} -> {Gabg}
(pseudo 4.) R3: {Gabg} + {LR} -> {LRGabg}

----------------------------------------------------------------------

An operation between pseudo reactions:
1.00 * R9 + 1.00 * R38 + 1.00 * R39 + 1.00 * R3

will result in empty reactant with zero mass:

:  -> {P}

This indicates a mass con

In [11]:
m.nodes

NodeView(({Pstat_sol}, {species_test}, {PstatDimer_nuc=PstatDimer_sol}, {Pstat_nuc=stat_nuc=stat_sol}))

In [11]:
som = m.getNode(simple.getMolecule("cf6p"))
print(som.identifier)

{cf6p=cfdp=cg1p=cg6p=cpg=crib5p=cribu5p=cxyl5p}


In [15]:
list(m.nodes)[4]

{cf6p=cfdp=cg1p=cg6p=cpg=crib5p=cribu5p=cxyl5p}

In [18]:
report, count = gr.reportReactionsInSOM(list(m.nodes)[4])
print(report)


1. vPGDH: cpg -> cribu5p;   {cpg=cribu5p}

2. vRu5P: cribu5p -> cxyl5p;   {cpg=cribu5p=cxyl5p}

3. vG6PDH: cg6p -> cpg;   {cpg=cribu5p=cxyl5p=cg6p}

4. vR5PI: cribu5p -> crib5p;   {cpg=cribu5p=cxyl5p=cg6p=crib5p}

5. vPGI: cg6p -> cf6p;   {cpg=cribu5p=cxyl5p=cg6p=crib5p=cf6p}

6. vPGM: cg6p -> cg1p;   {cpg=cribu5p=cxyl5p=cg6p=crib5p=cf6p=cg1p}

7. vPFK: cf6p -> cfdp;   {cpg=cribu5p=cxyl5p=cg6p=crib5p=cf6p=cg1p=cfdp}



In [6]:
type3_report, error_num = gr.reportTypeThreeError(m.type_three_errors, explain_details=True)
print(type3_report)
print(error_num)

We detected a mass imbalance from the following reactions:

1. R9: LRGabg -> LR + GaGTP + Gbg
2. R39: GaGDP + Gbg -> Gabg
3. R3: LR + Gabg -> LRGabg

----------------------------------------------------------------------

These uni-uni reactions created mass-equivalence.
(The molecules within a curly bracket have the same atomic mass.)

4. R26: GaGTP -> GaGDPP;   {GaGTP=GaGDPP}

----------------------------------------------------------------------

These multi-uni (uni-multi) reactions created mass-inequality.

5. R38: GaGDPP -> GaGDP + P

----------------------------------------------------------------------

Based on the reactions above, we have mass-equivalent pseudo reactions.

(pseudo 1.) R9: {LRGabg} -> {GaGDPP=GaGTP} + {Gbg} + {LR}
(pseudo 2.) R39: {GaGDP} + {Gbg} -> {Gabg}
(pseudo 3.) R3: {Gabg} + {LR} -> {LRGabg}

----------------------------------------------------------------------

An operation between pseudo reactions:

-1.00 * R9 - 1.00 * R39
 - 1.00 * R3


will result i

In [35]:
m.lower_inverse

,R30,R38,R32,R33,R18,R9,R35,R10,R23,R1,...,R19,R21,R22,R24,R27,R29,R31,R34,R36,R3
R30,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0
R38,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0
R32,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0
R33,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0
R18,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0
R9,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0
R35,0.0,0.0,0.0,-1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0
R10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0
R23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0
R1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0


In [37]:
print(m.convertReactionToSOMReaction(simple.getReaction("reaction_216")))
print(m.convertReactionToSOMReaction(simple.getReaction("reaction_124")))

reaction_216: {CamR_Ca2_BD_PP2B} -> {PP2B} + {CamR_Ca2_BD, CamT_Ca2_BD}
reaction_124: {CamR_Ca4_ABCD, CamT_Ca4_ABCD} -> {Ca} + {CamR_Ca3_ABD, CamT_Ca3_ABD}


In [13]:
"%.2f" % (3.00)

'3.00'

In [11]:
# this will test which model has which error;;
model_count = 0
count = 0
for i in range(1, 704):
  try:
    simple = load_file_from_curated_data(i)
    model_count += 1
    if model_count%100 == 0:
      print("we are doing model", model_count)
    m = GAMES_PP(simple)
    m.analyze(error_details=False)
    gr = GAMESReport(m)
    if m.echelon_errors:
      res, error_num = gr.reportEchelonError(m.echelon_errors, explain_details=True)
      if not res:
        count += 1
        print("model", model_count, " has a really wrong echelon error!")
        if count == 5:
          break
  except:
    pass

we are doing model 100
we are doing model 200
we are doing model 300
we are doing model 400
we are doing model 500
we are doing model 600
model 623  has a really wrong echelon error!


In [19]:
# this will test which model has which error;;
model_count = 0
count = 0
for i in range(1, 150):
  try:
    simple = load_file_from_curated_data(i)
    model_count += 1
    if model_count%100 == 0:
      print("we are doing model", model_count)
    m = GAMES_PP(simple)
    m.analyze(error_details=False)
    if m.echelon_errors or m.type_three_errors:
        print("model", model_count, " has an echelon or type three error!")
        if count == 5:
          break
  except:
    pass

model 14  has an echelon or type three error!
model 49  has an echelon or type three error!
model 51  has an echelon or type three error!
model 52  has an echelon or type three error!
we are doing model 100
model 102  has an echelon or type three error!
model 109  has an echelon or type three error!
model 140  has an echelon or type three error!
